In [ ]:
!pip install ninja
!pip3 install torch==1.10.2+cu113 torchvision==0.11.3+cu113 torchaudio==0.10.2+cu113 -f https://download.pytorch.org/whl/cu113/torch_stable.html



 

     |████████████████████████████████| 108 kB 5.5 MB/s 
Looking in links: https://download.pytorch.org/whl/cu113/torch_stable.html
     |██████████████▋                 | 834.1 MB 1.8 MB/s eta 0:09:15tcmalloc: large alloc 1147494400 bytes == 0x55a4e0cfc000 @  0x7f8fca1db615 0x55a4a770a3bc 0x55a4a77eb18a 0x55a4a770d1cd 0x55a4a77ffb3d 0x55a4a7781458 0x55a4a777c02f 0x55a4a770eaba 0x55a4a77812c0 0x55a4a777c02f 0x55a4a770eaba 0x55a4a777dcd4 0x55a4a7800986 0x55a4a777d350 0x55a4a7800986 0x55a4a777d350 0x55a4a7800986 0x55a4a777d350 0x55a4a770ef19 0x55a4a7752a79 0x55a4a770db32 0x55a4a77811dd 0x55a4a777c02f 0x55a4a770eaba 0x55a4a777dcd4 0x55a4a777c02f 0x55a4a770eaba 0x55a4a777ceae 0x55a4a770e9da 0x55a4a777d108 0x55a4a777c02f
     |██████████████████▌             | 1055.7 MB 1.4 MB/s eta 0:09:04tcmalloc: large alloc 1434370048 bytes == 0x55a525352000 @  0x7f8fca1db615 0x55a4a770a3bc 0x55a4a77eb18a 0x55a4a770d1cd 0x55a4a77ffb3d 0x55a4a7781458 0x55a4a777c02f 0x55a4a770eaba 0x55a4a77812c0 0x55a4a77

In [ ]:
!pip install ninja

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
from argparse import Namespace

import numpy as np
import torch
import sys
import dlib
sys.path.append(".")
sys.path.append("..")
sys.path.append('/content/drive/MyDrive/PSPNet/PSPNet')

from options.test_options import TestOptions
from models.psp import pSp
import cv2
from scripts.align_all_parallel import align_face
import torchvision.transforms as transforms

#test_opts = TestOptions().parse()

test_opts = Namespace(checkpoint_path=None, couple_outputs=False, data_path='gt_images', exp_dir=None, latent_mask=None, mix_alpha=None, n_images=None, n_outputs_to_generate=5, resize_factors=None, resize_outputs=False, test_batch_size=2, test_workers=2)


test_opts.checkpoint_path = '/content/drive/MyDrive/PSPNet/PSPNet/pretrained_models/psp_ffhq_encode.pt'


# update test options with options used during training
ckpt = torch.load(test_opts.checkpoint_path, map_location='cpu')
opts = ckpt['opts']
opts.update(vars(test_opts))
opts['output_size'] = 1024

opts = Namespace(**opts)



net = pSp(opts)
net.eval()
net.cuda()
#Prepare the PSPNet + StyleGAN network

transform = transforms.Compose([
	transforms.Resize(size=(256, 256)),
	transforms.ToTensor(),
	transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])]
)
#Preprocessing operations for each image

predictor = dlib.shape_predictor("/content/drive/MyDrive/PSPNet/PSPNet/shape_predictor_68_face_landmarks.dat")
#Predict facial landmarks

aligned_image = align_face(filepath="/content/drive/MyDrive/PSPNet/PSPNet/nuri.jpg", predictor=predictor)
aligned_image = aligned_image.convert("RGB")
# Align face using the landmarks

from_im = transform(aligned_image).unsqueeze(0)
# Do preprocessing on the aligned face

with torch.no_grad():
	input = from_im.float().cuda()
	_, latent_vector = net(input, randomize_noise=False, resize=False, return_latents=True)
	# Obtain features from input image
	np.save("/content/drive/MyDrive/PSPNet/PSPNet/res1",latent_vector.cpu())
	result, _ = net.decoder([latent_vector.float()],input_is_latent=True, randomize_noise=False, return_latents=False)
	# Feed features to the StyleGAN
	
	result = result.squeeze().permute((1,2,0)).cpu().numpy()
 
	

	result[result>1] = 1
	result[result<-1] = -1
	result = (255*(result+1)//2).astype(np.uint8)

	cv2.imwrite('/content/drive/MyDrive/PSPNet/PSPNet/test_1.png', result[:,:,[2,1,0]])


ModuleNotFoundError: ignored

In [ ]:
import os
from argparse import Namespace

import numpy as np
import torch
import sys
import dlib
sys.path.append(".")
sys.path.append("..")
sys.path.append('/content/drive/MyDrive/PSPNet/PSPNet')

from options.test_options import TestOptions
from models.psp import pSp
import cv2
from scripts.align_all_parallel import align_face
import torchvision.transforms as transforms

#test_opts = TestOptions().parse()

test_opts = Namespace(checkpoint_path=None, couple_outputs=False, data_path='gt_images', exp_dir=None, latent_mask=None, mix_alpha=None, n_images=None, n_outputs_to_generate=5, resize_factors=None, resize_outputs=False, test_batch_size=2, test_workers=2)


test_opts.checkpoint_path = '/content/drive/MyDrive/PSPNet/PSPNet/pretrained_models/psp_ffhq_encode.pt'


# update test options with options used during training
ckpt = torch.load(test_opts.checkpoint_path, map_location='cpu')
opts = ckpt['opts']
opts.update(vars(test_opts))
opts['output_size'] = 1024

opts = Namespace(**opts)



net = pSp(opts)
net.eval()
net.cuda()
#Prepare the PSPNet + StyleGAN network

transform = transforms.Compose([
	transforms.Resize(size=(256, 256)),
	transforms.ToTensor(),
	transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])]
)
#Preprocessing operations for each image

predictor = dlib.shape_predictor("/content/drive/MyDrive/PSPNet/PSPNet/shape_predictor_68_face_landmarks.dat")
#Predict facial landmarks

aligned_image = align_face(filepath="/content/drive/MyDrive/PSPNet/PSPNet/leo.jpg", predictor=predictor)
aligned_image = aligned_image.convert("RGB")
# Align face using the landmarks

from_im = transform(aligned_image).unsqueeze(0)
# Do preprocessing on the aligned face

with torch.no_grad():
	input = from_im.float().cuda()
	_, latent_vector = net(input, randomize_noise=False, resize=False, return_latents=True)
	# Obtain features from input image
	np.save("/content/drive/MyDrive/PSPNet/PSPNet/res2",latent_vector.cpu())
	result, _ = net.decoder([latent_vector.float()],input_is_latent=True, randomize_noise=False, return_latents=False)
	# Feed features to the StyleGAN
	
	result = result.squeeze().permute((1,2,0)).cpu().numpy()
 
	

	result[result>1] = 1
	result[result<-1] = -1
	result = (255*(result+1)//2).astype(np.uint8)

	cv2.imwrite('/content/drive/MyDrive/PSPNet/PSPNet/test_2.png', result[:,:,[2,1,0]])


In [ ]:
z1 = np.load('/content/drive/MyDrive/PSPNet/PSPNet/res1.npy')
z2 = np.load('/content/drive/MyDrive/PSPNet/PSPNet/res2.npy')

z1 = torch.from_numpy(z1).float().cuda()
z2 = torch.from_numpy(z2).float().cuda()
diff = z2-z1

diff = diff/240

l = []

result1, _ = net.decoder([z1.float()],input_is_latent=True, randomize_noise=False, return_latents=False)
result1 = result1.squeeze().permute((1,2,0)).cpu().detach().numpy()
 
	

result1[result1>1] = 1
result1[result1<-1] = -1
result1 = (255*(result1+1)//2).astype(np.uint8)



l.append(result1)
for i in range(240):
  z1=z1+diff
  result1, _ = net.decoder([z1.float()],input_is_latent=True, randomize_noise=False, return_latents=False)
  result1 = result1.squeeze().permute((1,2,0)).cpu().detach().numpy()

  result1[result1>1] = 1
  result1[result1<-1] = -1
  result1 = (255*(result1+1)//2).astype(np.uint8)

  l.append(result1)


In [ ]:
import moviepy.video.io.ImageSequenceClip as ImageSequenceClip

clip = ImageSequenceClip.ImageSequenceClip(l, 60)
clip.write_videofile("part2.mp4")

[MoviePy] >>>> Building video part2.mp4
[MoviePy] Writing video part2.mp4


100%|██████████| 241/241 [00:15<00:00, 15.47it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: part2.mp4 

